In [1]:
# Importing required liabraries

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set up the WebDriver
driver = webdriver.Chrome()
driver.get('https://njdg.ecourts.gov.in/scnjdg/')

In [3]:
# Find and click the link to open the popup window
link = driver.find_element(By.XPATH, '//a[contains(@onclick, "fetchCasesRecords")]')
link.click()

In [4]:
# Wait for the popup window to appear
wait = WebDriverWait(driver, 10)
popup = wait.until(EC.presence_of_element_located((By.ID, 'modal_case_data')))


In [5]:
# Checking dropdown element
dropdown = driver.find_element(By.NAME, 'tbl_cases_length')
# Click the dropdown to open options
dropdown.click()

In [6]:
# Waiting for the dropdown options to be visible
wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//select[@name="tbl_cases_length"]/option')))

# Finding and clicking the option with value "1000"
option_1000 = driver.find_element(By.XPATH, '//select[@name="tbl_cases_length"]/option[@value="1000"]')
option_1000.click()

In [7]:
# Checking for table data 
table = driver.find_element(By.ID, 'tbl_cases_report_body')
table

<selenium.webdriver.remote.webelement.WebElement (session="b1bc9b16a162321586d5d4276954f986", element="91FADA9B4530089204A454E91E14A3F2_element_33")>

In [8]:
# Extracting rows
rows = table.find_elements(By.TAG_NAME, 'tr')

In [9]:
# Placeholder for data collection
all_data = []

while True:
    # Wait for the table to load
    table = driver.find_element(By.ID, 'tbl_cases_report_body')

    # Extract rows from the table
    rows = table.find_elements(By.TAG_NAME, 'tr')

    # Extract data from each row and store in a list of dictionaries
    for row in rows:
        columns = row.find_elements(By.TAG_NAME, 'td')
        try:
            data = {
                'Diary Number': columns[0].text.strip(),
                'Case Number': columns[1].text.strip(),
                'Date of Filing': columns[2].text.strip(),
                'Party Name': columns[3].text.strip()
            }
            all_data.append(data)
        except IndexError:
            # If the row doesn't have expected columns, continue to the next row
            continue

    # Checking if the "Next" button is disabled
    next_button = driver.find_element(By.ID, 'tbl_cases_next')
    if 'disabled' in next_button.get_attribute('class'):
        break

    # Clicking the "Next" button to move to the next page
    next_button.click()

    # Wait for a short time to load the next page
    time.sleep(4)

# Creating a DataFrame
df = pd.DataFrame(all_data)

# Display the DataFrame
df

,Diary Number,Case Number,Date of Filing,Party Name
0,41091/2023,Unregistered,04-10-2023,UPENDRA NATH DALAI Vs ALL INDIA PRESIDENT BHAR...
1,40882/2023,Unregistered,03-10-2023,ROHAN CHATTERJEE Vs THE STATE OF WEST BENGAL
2,41501/2023,Unregistered,06-10-2023,SOMAKKA DECEASED BY LRS Vs ANAND @ ADIVEPPA
3,41949/2023,Unregistered,09-10-2023,RAKESH KAUSHIK Vs THE STATE OF HARYANA
4,40644/2023,SPECIAL LEAVE PETITION (CIVIL) / 22561 / 2023,02-10-2023,SUDEEPTA BISWAJIT GANGULY Vs BISWAJIT BIJANKUM...
...,...,...,...,...
3183,44362/2023,Unregistered,21-10-2023,PUNAM LALWANI Vs PRATIBHA SONI
3184,43514/2023,Unregistered,18-10-2023,COMMISSIONER OF SALES TAX Vs M/S. BIRLA TYRES
3185,41271/2023,WRIT PETITION (CIVIL) / 1147 / 2023,05-10-2023,E. R. GOPAKUMAR Vs THE STATE OF TAMIL NADU
3186,41772/2023,SPECIAL LEAVE PETITION (CIVIL) / 24804 / 2023,09-10-2023,MANAGING DIRECTOR U.P. JAL NIGAM (URBAN) LUCK ...


In [10]:
driver.quit()

In [11]:
df.to_csv('Raw_SC_Civil_Cases_Scraped_Data.csv', index= False)